In [16]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import binance as bn
import pandas as pd
import sys
sys.path.append('/Users/jp/Desktop/Investment/utils')
import utils as utils
from datetime import datetime

In [17]:
# api_key = "y6D1BHRcjTDCcaOi21C3LuwHpYxe0itpoOYca1DTYDlFNrNBi5OZkeFGK4rBb4Ga"
# api_secret = "c4pIQ5H5s9d1ezmTrfxAfguBTC4f6QFLoBEK4ftokm9yEPBbVyqDF05dxCbTnr57"

In [18]:
# client = Client(api_key=api_key, api_secret=api_secret, tld = "com", testnet = True)

In [36]:
class Long_macd_trader():
    
    """Class to perform live testing using Binance testnet stream of data
    """
    def __init__(self, symbol, position, interval, start, end=None, testnet=True):
        
        """Long_macd_trader class constructor
        :param interval: a string among the followings: ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        :type interval: str.
        :param start: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-01-29-20:00"
        :type start: str.
        :param end: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-02-29-20:00"
        :type end: str.
        :position: initial position in the market. 1 for long (have the asset) 0 for neutral (dont have the asset)
        :position: integer
        :back_testing: set to True if backtesting is necessary
        """
        self.symbol = symbol
        self.position = position        
        self.interval = interval
        self.start = start
        self.end = end
        self.testnet= testnet
        self.data = pd.DataFrame()
    
    def stream_candles(self, msg):
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print("Time: {} | Price: {}".format(event_time, close))
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
    def start_trading(self):

        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        self.get_recent_data()
        self.twm.start_kline_socket(callback = self.stream_candles, symbol = self.symbol, interval = self.interval)
        
    def stop_ses(self):
        self.twm.stop()
    
    def define_strategy(self):
        pass
        
    def execute_trades(self):
        pass
        
    def reporting(self):
        pass
        
    def get_recent_data(self):
        '''
        REMARK: Introduced time must be in Tokyo time (UTC+9) but the calculations will be in UTC
        Prepare all the fields of data necessary for the study. The interval of dates to be studied is the one
        given when delclaring the class. To prepare another interval of dates, please create another class instance.
        :param start: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-01-29-20:00"
        :type start: str.
        :param end: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-02-29-20:00"
        :type end: str.
        :param interval: string among the followings: ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        :type interval: str.
        '''
        from_time = int(datetime.strptime(self.start, "%Y-%m-%d-%H:%M").timestamp()*1000)
        if (self.end != None):      
            to_time = int(datetime.strptime(self.end, "%Y-%m-%d-%H:%M").timestamp()*1000)
        else:
            to_time=self.end
            
        self.data = utils.get_history_v3(symbol=self.symbol, interval=self.interval, start=from_time, end=to_time, testnet=self.testnet)[0]
        self.data["Complete"] = [True for row in range(len(self.data)-1)] + [False]
    

In [37]:
trader_test = Long_macd_trader(symbol="BTCUSDT", position=0, interval='1h', start='2022-08-29-20:00', end=None)

In [38]:
trader_test.start_trading()

265
Time: 2022-09-18 07:57:01.304000 | Price: 20050.57
Time: 2022-09-18 07:57:03.365000 | Price: 20050.49


In [39]:
trader_test.stop_ses()

Time: 2022-09-18 07:57:05.404000 | Price: 20050.92


In [32]:
trader_test.data_hist

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2022-09-07 07:00:00,19107.80,20554.27,15668.40,18801.89,3.365345,True
2022-09-07 08:00:00,18793.68,18855.56,17311.28,18806.14,248.350868,True
2022-09-07 09:00:00,18806.51,18815.41,18043.65,18788.81,327.160458,True
2022-09-07 10:00:00,18789.51,18820.60,18573.63,18747.98,332.715797,True
2022-09-07 11:00:00,18747.98,18774.11,18698.85,18740.17,351.983210,True
...,...,...,...,...,...,...
2022-09-18 03:00:00,20033.03,20036.97,19953.85,19995.39,310.158444,True
2022-09-18 04:00:00,19995.51,20473.27,19955.25,20011.17,309.506721,True
2022-09-18 05:00:00,20011.17,20487.06,19286.42,20071.51,280.597889,True
